### Расчёт электрического поля системы зарядов

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from ipywidgets import widgets

    
def u(x_charges, y_charges, q_charges, x, y):
    x_charges, y_charges = np.array(x_charges), np.array(y_charges)
    q_charges = np.array(q_charges)
    l = (np.sqrt((x - x_charges)**2 + (y - y_charges)**2))
    return np.sum(np.log(l)*q_charges)
    

def data_parse(data):
    data_L = eval(data)
    x_list = [i[0] for i in data_L]
    if min(x_list) < -10 or max(x_list) > 10:
        raise ValueError('x from [-10, 10]')
    y_list = [i[1] for i in data_L]
    if min(y_list) < -10 or max(y_list) > 10:
        raise ValueError('y from [-10, 10]')
    q_list = [i[2] for i in data_L]
    if min(q_list) < -10 or max(q_list) > 10:
        raise ValueError('q from [-10, 10]')
    return x_list, y_list, q_list


def calc_field(x_list, y_list, q_list, shape):
    xx = np.linspace(-20, 20, shape)
    yy = np.linspace(-20, 20, shape)
    res = np.zeros((shape, shape))
    for i in range(len(xx)):
        for j in range(len(yy)):
            uu = u(x_list, y_list, q_list, xx[i], yy[j])
            res[i][j] = int(uu)
    return res, xx, yy

def save_fig(fig, name):
    def inner(*args):
        fig.savefig(name)
    return inner
        

def main(*args):
    data = w_data.value
    cout_line = w_count.value
    x_list, y_list, q_list = data_parse(data)
    count = len(x_list)
    cmap1 = w_cmapf.value
    cmap2 = w_cmap.value
    with out2:
        clear_output(wait=True)
        field, xx, yy = calc_field(x_list, y_list, q_list, 50)
        grad = np.gradient(field)
        fig = plt.figure(figsize=(5, 5))
        ax1 = fig.add_subplot(111)
        levels = cout_line
        img = ax1.contourf(xx, yy, field, cmap=cmap1, levels=levels)
        plt.colorbar(img)
        ax1.set_xlabel('x')
        ax1.set_ylabel('y')
        ax1.set_title('Распределение потенциалов')
        cs = ax1.contour(xx, yy, field, cmap=cmap2, levels=levels)
        ax1.streamplot(xx, yy, grad[1], grad[0])
        ax1.clabel(cs, fontsize=10, fmt='%.1f')
        plt.tight_layout()
        plt.show()
        save = save_fig(fig, data)
        button = widgets.Button(description='Сохранить', button_style='primary')
        button.on_click(save)
        display(button)

test = '[(0, 10, 10), (10, -10, 5), (5, -8, 8)]'
out1 = widgets.Output(layout={'width': '50%'})
out2 = widgets.Output(layout={'width': '50%'})
w_data_label = widgets.Label('Параметры зарядов')
w_data = widgets.Textarea(placeholder='[(x0, y0, q0), (x1, y2, q3)]', value=test)
w_count_label = widgets.Label('Кол-во эквипотенциалей')
w_count =  widgets.IntSlider(min=1, max=30, step=1, value=20)
w_cmapf_label = widgets.Label('Палитра заливки')
w_cmapf = widgets.RadioButtons(options={'gray': 'gray',
                                        'jet': 'jet',
                                         'plasma': 'plasma',
                                         'inferno': 'inferno'})
w_cmap_label = widgets.Label('Палитра линий')
w_cmap = widgets.RadioButtons(options={'jet': 'jet',
                                         'gray': 'gray',
                                         'plasma': 'plasma',
                                         'inferno': 'inferno'})
button = widgets.Button(description='Смоделировать', button_style='primary')
button.on_click(main)
display(widgets.HBox([out1, out2]))
with out1:
    display(w_data_label, w_data, w_count_label, w_count,
            w_cmapf_label, w_cmapf, w_cmap_label, w_cmap, button)